In [ ]:
import os
DATADIR = "/Users/markoni/Desktop/try_letters/(letters)"

TESTDIR = "/Users/markoni/Desktop/try_letters/TEST_DATA"
pref='_low'
for letter in os.listdir(DATADIR):
    os.rename(os.path.join(DATADIR,letter), os.path.join(DATADIR,letter+pref))
    
       
        
            

In [ ]:
import os

for dr in os.listdir(DATADIR):
    if dr[0]!='.':
        path = os.path.join(DATADIR,dr)
        testpath = os.path.join(TESTDIR,dr)
        if not os.path.exists(os.path.join(TESTDIR,dr)):
            os.makedirs(os.path.join(TESTDIR,dr))
        for img in os.listdir(path):
            if int(img[-6:-4])==0:
                os.rename(os.path.join(path,img), os.path.join(testpath,img))


In [ ]:
from tqdm import tqdm

In [ ]:
import os
import cv2


training_data = [] 

def create_training_data():
    for category in CATEGORIES:  

        path = os.path.join(DATADIR,category)  
        class_num = CATEGORIES.index(category)  

        for img in tqdm(os.listdir(path)):  
            try:
                img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_GRAYSCALE) 
                new_array = cv2.resize(img_array, (70, 70))
                training_data.append([new_array, class_num])  
            except Exception as e:  
                print('error')
                pass
           

        
CATEGORIES=[]
for letter in os.listdir(DATADIR):
    if letter[0]!='.':
        CATEGORIES.append(letter)
        
create_training_data()

In [ ]:
DATAPICKLETEST = "/Users/markoni/Desktop/try_letters/INPUTS-TEST.pickle"
LABELSPICKLETEST = "/Users/markoni/Desktop/try_letters/LABELS-TEST.pickle"

In [ ]:
import random

random.shuffle(training_data)

inputs = []
labels = []

for inp,label in training_data:
    inputs.append(inp)
    labels.append(label)
    
    
import pickle

pickle_out = open(DATAPICKLETEST,"wb")
pickle.dump(inputs, pickle_out)
pickle_out.close()

pickle_out = open(LABELSPICKLETEST,"wb")
pickle.dump(labels, pickle_out)
pickle_out.close()

In [ ]:
import tensorflow as tf
import numpy
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils

In [ ]:
pickle_in = open("/content/drive/LetsDoThis/INPUTS.pickle","rb")
X = pickle.load(pickle_in)
X = numpy.array(X).reshape(-1, 70, 70, 1)
X = (1-X/255)
pickle_in = open("/content/drive/LetsDoThis/LABELS.pickle","rb")
y = pickle.load(pickle_in)

In [ ]:
y=np_utils.to_categorical(y)
model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=X.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  
model.add(Dense(128))

model.add(Dense(61))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X, y, batch_size=32, validation_split=0.1)

In [ ]:
model.save('/content/drive/LetsDoThis/cap_let_reader.model')
MODEL = '/content/drive/LetsDoThis/cap_let_reader.model'

In [ ]:
import tensorflow as tf

new_model = tf.keras.models.load_model(MODEL)

pickle_in = open(INPUTS_TEST,"rb")
XT = pickle.load(pickle_in)
XT = numpy.array(XT).reshape(-1, 70, 70, 1)
pickle_in = open(LABELS_TEST,"rb")
yT = pickle.load(pickle_in)

XT = (1.0 - XT/255.0)
yT=tf.keras.utils.to_categorical(yT)

predictions = new_model.predict(XT)


In [ ]:
import numpy as np
errs = []
for i in range(len(yT)):
    if np.argmax(yT[i])!=np.argmax(predictions[i]):
        errs.append(i)